In [1]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

/home/taehyung/anaconda3/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/taehyung/anaconda3/envs/llama/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.7/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/taehyung/anaconda3/envs/llama/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/taehyung/anaconda3/envs/llama/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/taehyung/anaconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/taehyung/anaconda3/envs/llama/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda-11.6/lib64')}
  warn(msg)


In [2]:
from peft import PeftModel, PeftConfig


peft_model_id = './tmp/fashiongen/13B'

config = PeftConfig.from_pretrained(peft_model_id)
model = LlamaForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = LlamaTokenizer.from_pretrained(config.base_model_name_or_path)

Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.31s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [3]:
eval_prompt = """
Using the given fashion attributes: "black, graphic, t-shirt" please provide a detailed description
---
Description: 
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Using the given fashion attributes: "black, graphic, t-shirt" please provide a detailed description
---
Description: 
Short sleeve cotton jersey t-shirt in black. Rib knit crewneck collar. Graphic printed in white at front. Tonal stitching.


In [9]:
import pandas as pd

val_data = pd.read_csv('./val.csv')

prompt = (
    f"Using the given fashion attributes: {{attribute}}, please provide a detailed description\n---\nDescription: "
)


val_data

,Unnamed: 0,id,attribute,text
0,0,0,"faded, jacket, blue",Denim-like jogg jacket in blue. Fading and whi...
1,1,1,"jeans, light, distressed, slim, grey",Slim-fit jeans in light grey. Distressing and ...
2,2,2,"leather, jacket, suede, black, paneled",Long sleeve suede jacket in black. Tonal grain...
3,3,3,"jeans, cool, distressed, blue, guy",Slim-fit jeans in dark blue. Distressing throu...
4,4,4,"navy, classic, trousers",Slim-fit trousers in navy. Four-pocket styling...
...,...,...,...,...
7514,7514,7514,"car, beige, coat",Long sleeve cotton gabardine coat in beige and...
7515,7515,7515,"navy, coat",Long sleeve waterproof rubberized cotton rainc...
7516,7516,7516,"leather, platform, double, black, espadrilles",Grained leather espadrilles in black. Round to...
7517,7517,7517,"denim, black, jeans",Slim-fit jeans in black. High-rise. Five-pocke...


In [13]:
def make_prompt(sample):
    return prompt.format(attribute=sample)


In [16]:
from tqdm import tqdm

val_prompt = list(map(make_prompt, list(val_data['attribute'])))

model_inputs = [0 for i in range(len(val_prompt))]


for i in tqdm(range(len(val_prompt))):
    model_input = tokenizer(val_prompt[i], return_tensors='pt').to('cuda')
    
    model_inputs[i]=model_input

100%|██████████| 7519/7519 [00:03<00:00, 1884.66it/s]


In [ ]:
gen_list = [0 for i in  range(len(model_inputs))]

In [ ]:
model.eval()
with torch.no_grad():
    for i in tqdm(range(len(model_inputs))):
        if gen_list[i] != 0:
            continue
        else:
            gen = tokenizer.decode(model.generate(**model_inputs[i], max_new_tokens=100)[0], skip_special_tokens=True).split(':')[-1].strip()
            gen_list[i] = gen


 43%|████▎     | 3270/7519 [17:22<6:47:27,  5.75s/it] 

In [42]:
gen_3369 = [i for i in gen_list if i != 0]
real_data = list(val_data['text'][:3369])

print(len(gen_3369))
print(len(real_data))

3369
3369


In [46]:
from transformers import AutoTokenizer, BertModel
import torch
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")


gen_embedding = np.zeros((3369,768))

for i in tqdm(range(3369)):
    text = gen_3369[i]    
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    vector = outputs.pooler_output
    
    gen_embedding[i] = vector.detach().numpy()

100%|██████████| 3369/3369 [01:14<00:00, 44.95it/s]


In [47]:
real_embedding = np.zeros((3369, 768))

for i in tqdm(range(3369)):
    text = real_data[i]    
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    vector = outputs.pooler_output
    
    real_embedding[i] = vector.detach().numpy()

100%|██████████| 3369/3369 [01:19<00:00, 42.40it/s]


In [67]:
from sklearn.metrics.pairwise import cosine_similarity


results = [0 for i in range(3369)]

for i in tqdm(range(3369)):
    sim_result = cosine_similarity(gen_embedding[i].reshape(1,-1), real_embedding[i].reshape(1,-1))[0][0]
    results[i] = sim_result

100%|██████████| 3369/3369 [00:00<00:00, 3448.25it/s]


In [74]:
gen_3369[:10]

['Long sleeve cotton twill jacket in blue. Fading throughout. Spread collar. Button closure at front. Flap pockets at chest. Single-button barrel cuffs. Buttoned cinch tabs at back hem. Buttoned cinch tabs at cuffs. Tonal stitching.',
 '12 oz. slim-fit jeans in light grey. Distressing throughout. Five-pocket styling. Leather logo patch at back waistband. Zip-fly. Tonal stitching. Approx. 7" leg opening.',
 'Long sleeve suede jacket in black. Notched lapel collar. Off-set zip closure at front. Zippered pockets at waist. Zippered sleeve cuffs. Fully lined. Tonal stitching.',
 '12 oz. Japanese selvedge denim jeans in blue. Fading, distressing, and whiskering throughout. Five-pocket styling. Leather logo patch at back waistband. Zip-fly. Contrast stitching in tan. Approx. 8.5" leg opening.',
 '10 oz. cotton twill trousers in navy. Four-pocket styling. Zip-fly. Tonal stitching.',
 'Long sleeve cotton shirt in red featuring plaid pattern in tones of red, black, and white throughout. Spread c

In [76]:
real_data[:10]

['Denim-like jogg jacket in blue. Fading and whiskering throughout. Spread collar. Copper tone button closures at front. Flap pockets at chest with metallic logo plaque. Seam pockets at sides. Cinch tabs at back waistband. Single button sleeve cuffs. Tone on tone stitching.',
 'Slim-fit jeans in light grey. Distressing and fading throughout. Seven-pocket styling. Textured black leather logo patch at back waist. Tonal stitching. Red logo tab at button-fly.',
 'Long sleeve suede jacket in black. Tonal grained leather paneling throughout. Stand collar. Zip closure and zippered welt pockets at front. Zippered vents at back hem. Welt pockets at interior. Fully lined. Tonal stitching. Zippered expansion panels at sleeve cuffs.',
 'Slim-fit jeans in dark blue. Distressing throughout. Fading at front. Textured black leather logo patch at back waist. Silver-tone metal logo plaque at back pocket. Contrast stitching in tan. Red logo tab at button-fly.',
 'Slim-fit trousers in navy. Four-pocket st